In [24]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from xgboost import XGBClassifier


In [25]:
data_7th=pd.read_csv("New7th.csv")
data_7th


,Marathi,Urdu,Hindi,English,History,Science,Geography,Drawing,Sports,Environmental Studies,Math,Computer,Interest,Recommended Career
0,74,46,74,35,74,64,74,53,28,20,74,67,"Hindi, History, Geography, Math","Meteorologist, Genealogist, Literature and Edu..."
1,54,90,69,54,72,90,22,79,90,79,65,25,"Urdu, Science, Sports","Sports Nutritionist/Dietitian, Professional At..."
2,59,68,22,82,60,82,74,71,55,79,82,82,"English, Science, Math, Computer","Literature and Education, Corporate and Busine..."
3,31,61,40,25,26,93,93,93,38,93,24,60,"Science, Geography, Drawing, Environmental Stu...","Meteorologist, Concept Artist, Set Designer/Il..."
4,52,25,22,21,59,35,44,53,62,45,24,76,Computer,Artificial Intelligence (AI) / Machine Learnin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,71,79,27,23,40,54,20,49,56,59,31,73,Urdu,"Literature and Education, Government and Admin..."
9996,57,56,77,72,23,22,43,43,25,23,41,21,Hindi,"Literature and Education, Government and Admin..."
9997,22,75,69,33,25,74,35,44,80,30,53,49,Sports,"Sports Nutritionist/Dietitian, Professional At..."
9998,26,20,72,58,25,58,46,46,80,35,75,51,Sports,"Sports Nutritionist/Dietitian, Professional At..."


In [26]:
data_7th.dtypes

Marathi                   int64
Urdu                      int64
Hindi                     int64
English                   int64
History                   int64
Science                   int64
Geography                 int64
Drawing                   int64
Sports                    int64
Environmental Studies     int64
Math                      int64
Computer                  int64
Interest                 object
Recommended Career       object
dtype: object

In [27]:
new_7th = data_7th.drop(['Interest', 'Recommended Career'], axis=1)
new_7th

,Marathi,Urdu,Hindi,English,History,Science,Geography,Drawing,Sports,Environmental Studies,Math,Computer
0,74,46,74,35,74,64,74,53,28,20,74,67
1,54,90,69,54,72,90,22,79,90,79,65,25
2,59,68,22,82,60,82,74,71,55,79,82,82
3,31,61,40,25,26,93,93,93,38,93,24,60
4,52,25,22,21,59,35,44,53,62,45,24,76
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,71,79,27,23,40,54,20,49,56,59,31,73
9996,57,56,77,72,23,22,43,43,25,23,41,21
9997,22,75,69,33,25,74,35,44,80,30,53,49
9998,26,20,72,58,25,58,46,46,80,35,75,51


In [28]:
new_7th.insert(0, 'Unique_ID', [f"stud_{i}" for i in range(1, len(new_7th) + 1)])
new_7th

,Unique_ID,Marathi,Urdu,Hindi,English,History,Science,Geography,Drawing,Sports,Environmental Studies,Math,Computer
0,stud_1,74,46,74,35,74,64,74,53,28,20,74,67
1,stud_2,54,90,69,54,72,90,22,79,90,79,65,25
2,stud_3,59,68,22,82,60,82,74,71,55,79,82,82
3,stud_4,31,61,40,25,26,93,93,93,38,93,24,60
4,stud_5,52,25,22,21,59,35,44,53,62,45,24,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,stud_9996,71,79,27,23,40,54,20,49,56,59,31,73
9996,stud_9997,57,56,77,72,23,22,43,43,25,23,41,21
9997,stud_9998,22,75,69,33,25,74,35,44,80,30,53,49
9998,stud_9999,26,20,72,58,25,58,46,46,80,35,75,51


In [29]:
# Ensure all subject columns are numeric
new_7th.iloc[:, 1:] = new_7th.iloc[:, 1:].apply(pd.to_numeric, errors="coerce")

# Function to find top and second-top subjects excluding Unique_ID
def find_top_interests(row):
    # Exclude the Unique_ID column
    subject_marks = {subject: mark for subject, mark in row.items() if subject != "Unique_ID"}
    
    # Find the highest and second-highest marks
    unique_marks = sorted(set(subject_marks.values()), reverse=True)
    if len(unique_marks) >= 2:
        top_marks = unique_marks[:2]  # Top two unique marks
    elif len(unique_marks) == 1:
        top_marks = unique_marks  # Only one unique mark
    else:
        top_marks = []  # No marks present
    
    # Collect all subjects with marks in top_marks
    top_subjects = [subject for subject, mark in subject_marks.items() if mark in top_marks]
    return ", ".join(top_subjects)

# Apply the function to find the Interest column
new_7th["Interest"] = new_7th.apply(find_top_interests, axis=1)
new_7th
file_path = "Latest7th.csv"  # Specify your desired file path
new_7th.to_csv(file_path, index=False)  # Save the DataFrame without the index
print(f"CSV file saved")

CSV file saved


In [42]:
# comb_5th_6th=pd.read_csv("combination_5th_6th.csv")
# comb_5th_6th